In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re

In [ ]:
optima = {
    "br17":     39,
    "ft53":   6905,
    "ft70":  38673,
    "ftv33":  1286,
    "ftv35":  1473,
    "ftv38":  1530,
    "ftv44":  1613,
    "ftv47":  1776,
    "ftv55":  1608,
    "ftv64":  1839,
    "ftv70":  1950,
    "ftv90":  1579,
    "ftv100": 1788,
    "ftv110": 1958,
    "ftv120": 2166,
    "ftv130": 2307,
    "ftv140": 2420,
    "ftv150": 2611,
    "ftv160": 2683,
    "ftv170": 2755,
    "kro124p":36230,
    "p43":    5620,
    "rbg323": 1326,
    "rbg358": 1163,
    "rbg403": 2465, 
    "rbg443": 2720,
    "ry48p": 14422,
}

In [ ]:
def annotate_ordering(df: pd.DataFrame) -> pd.DataFrame:
    df["instance_size"] = df.instance.apply(lambda x: int(re.search(r'\d+', x).group()))
    df["method_idx"] = df["method"].apply(lambda x: all_methods.index(x))
    return df.sort_values(by=['instance_size', 'method_idx'])

all_res_df = pd.read_json('../data/results/atsp-out-300.json')

selected_instances = all_res_df['instance'].unique()

all_methods = [
    "steepest-search",
    "steepest-search-nn",
    "greedy-search",
    "greedy-search-nn",
    "nn-heuristic",
    "random-walk",
    "random"
]
selected_methods = [
    "steepest-search",
    "greedy-search",
    "nn-heuristic",
    "random-walk",
    "random"
]

all_res_df = annotate_ordering(all_res_df).reset_index(drop=True)
all_res_df.head()

In [ ]:
LS_METHODS = ["steepest-search", "greedy-search"]
LS_NN_METHODS = ["steepest-search-nn", "greedy-search-nn"]
RANDOM_METHODS = ["random-walk", "random"]

In [ ]:
COLORS = dict(zip(selected_methods, sns.palettes.SEABORN_PALETTES["muted"]))
COLORS["steepest-search"] = "#6a9ec3"
COLORS["steepest-search-nn"] = "#14397E"
COLORS["greedy-search-nn"] = "#76330E"

all_res_df["distance"] = all_res_df.apply(lambda row: (row['cost'] - optima[row['instance']]) / optima[row['instance']], axis=1)
groupped_df = all_res_df.groupby(['instance', 'method'])[["distance", "time", "iterations", "steps", "evaluations", 'cost']].agg(['mean', 'std', 'min', 'max'])
groupped_df.columns = ["_".join(x) for x in groupped_df.columns]
stats_df = groupped_df.reset_index()
stats_df = annotate_ordering(stats_df).reset_index(drop=True)
stats_df.head()

In [ ]:
def plot_violin_by_attr(
        res_df: pd.DataFrame, 
        selected_methods: list[str],
        selected_instances: list[str], 
        attribute: str,
        ax: plt.Axes,
    ):
    filtered_df = res_df[res_df['method'].isin(selected_methods) & res_df['instance'].isin(selected_instances)]
    sns.violinplot(
        x='instance', 
        y=attribute, 
        hue='method', 
        data=filtered_df, 
        ax=ax,
        linewidth=0, 
        scale='width', 
        palette=COLORS, 
    )
    ax.vlines(
        np.arange(-1, len(selected_instances)) + 0.5, 
        0, 
        filtered_df[attribute].max(), 
        linestyles='dotted', 
        colors='gray', 
        alpha=0.5,
    )
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    return ax


def plot_bars(
    res_df: pd.DataFrame,
    selected_methods: list[str],
    selected_instances: list[str],
    attribute: str,
    ax: plt.Axes,
    err_attribute: str = None,
):
    filtered_df = res_df[res_df['method'].isin(selected_methods) & res_df['instance'].isin(selected_instances)]
    sns.barplot(
        x='instance', 
        y=attribute, 
        hue='method', 
        data=filtered_df, 
        ax=ax,
        palette=COLORS,
    )
    if err_attribute:
        x_coords = np.array([p.get_x() + p.get_width() / 2 for p in ax.patches])
        y_coords = np.array([p.get_height() for p in ax.patches])
        sorted_idx = np.argsort(x_coords)
        x_coords = x_coords[sorted_idx]
        y_coords = y_coords[sorted_idx]
        ax.errorbar(x_coords, y_coords, yerr=filtered_df[err_attribute].to_numpy(), fmt='none', ecolor='black', capsize=2)


    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    return ax

In [ ]:
sns.set_style("ticks")

## 1. Quality (distance from the optimum)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
plot_violin_by_attr(all_res_df, [*LS_METHODS, *LS_NN_METHODS, "nn-heuristic"], selected_instances, "distance", axs[0])
plot_violin_by_attr(all_res_df, RANDOM_METHODS, selected_instances, "distance", axs[1])
axs[0].set_xlabel("")
axs[0].set_ylabel("Distance from optimal solution")
axs[1].set_ylabel("Distance from optimal solution")
axs[1].set_xlabel("Instance")
axs[1].legend()

plt.tight_layout()
plt.savefig("plots/1/violin_distance.png")
plt.show()

In [ ]:
with sns.axes_style("whitegrid"):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
    plot_bars(stats_df, selected_methods, selected_instances, "distance_mean", axs[0], "distance_std")
    axs[0].set_ylabel("Distance to optimum")
    axs[0].set_xlabel("Instance")
    axs[0].set_title("Mean")
    plot_bars(stats_df, selected_methods, selected_instances, "distance_min", axs[1])
    axs[1].legend().remove()
    axs[1].set_ylabel("")
    axs[1].set_xlabel("Instance")
    axs[1].set_title("Min")
    plt.yscale('log')
    plt.tight_layout()
    plt.savefig("plots/1/bar_distance.png")
    plt.show()

# 2. Time

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
plot_violin_by_attr(all_res_df, ["steepest-search", "greedy-search", "nn-heuristic"], selected_instances, "time", ax)

ax.legend()
ax.set_ylabel("Running Time")
ax.set_xlabel("Instance")
plt.yscale('log')
plt.yticks([10**i for i in range(3, 13, 3)], ["1 us", "1 ms", "1 s", "1000 s"])
plt.tight_layout()
plt.savefig("plots/1/violin_time.png")
plt.show()

In [ ]:
with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    plot_bars(stats_df, selected_methods + LS_NN_METHODS, selected_instances, "time_mean", ax, "time_std")
    ax.legend(
        loc='center left',
        bbox_to_anchor=(1, 0.5),
    )

    ax.set_ylabel("Running Time")
    ax.set_xlabel("Instance")
    plt.yscale('log')
    plt.yticks([10**i for i in range(0, 13, 3)], ["1 ns", "1 us", "1 ms", "1 s", "1000 s"])
    plt.tight_layout()
    plt.savefig("plots/1/bar_time.png")
    plt.show()

In [ ]:
# plot mean time over instance size for each method
with sns.axes_style("whitegrid"):
    fig, axs = plt.subplots(1, 2, figsize=(16, 4))
    sns.lineplot(
        x='instance_size',
        y='time_mean',
        hue='method',
        data=stats_df[stats_df['method'].isin(selected_methods)],
        ax=axs[0],
        palette=COLORS,
        marker="o"
    )
    axs[0].set_ylabel("Running Time")
    axs[0].set_xlabel("Instance size")
    axs[0].set_yscale('log')
    axs[0].set_yticks([10**i for i in range(0, 13, 3)], ["1 ns", "1 us", "1 ms", "1 s", "1000 s"])
    axs[0].legend().remove()
    # plot mean distance over instance size for each method
    # highligh points as dots

    sns.lineplot(
        x='instance_size',
        y='distance_mean',
        hue='method',
        data=stats_df[stats_df['method'].isin(selected_methods)],
        ax=axs[1],
        palette=COLORS,
        marker="o"
    )
    axs[1].set_ylabel("Distance")
    axs[1].set_xlabel("Instance size")

    # plt.yscale('log')
    plt.tight_layout()
    plt.savefig("plots/1/line_time_distance.png")
    plt.show()

# 3. Efficiency of algorithms

In [ ]:
# plot mean time over mean distance for each method
fig, axs = plt.subplots(2, 4, figsize=(12, 6))#, sharex=True, sharey=True)

for instance, ax in zip(selected_instances, axs.flatten()):
    subset_df = all_res_df[(all_res_df['instance'] == instance) & (all_res_df['method'].isin(selected_methods + LS_NN_METHODS))]
    sns.scatterplot(
        x='time',
        y='distance',
        hue='method',
        data=subset_df,
        ax=ax,
        alpha=0.3,
        palette=COLORS,
    )
    summary_subset_df = stats_df[(stats_df['instance'] == instance) & (stats_df['method'].isin(selected_methods + LS_NN_METHODS))]

    ax.set_xlabel("Mean Time [ns]")
    ax.set_ylabel("Mean Distance")
    ax.set_title(instance)
    ax.set_yscale('log')
    ax.get_legend().remove()

plt.tight_layout()
plt.savefig("plots/1/efficiency.png")
plt.show()

# 4. Number of steps

In [ ]:
with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    plot_bars(stats_df, LS_METHODS + LS_NN_METHODS, selected_instances, "steps_mean", ax, "steps_std")
    ax.set_ylabel("Number of steps")
    ax.set_xlabel("Instance")
    plt.tight_layout()
    plt.yscale('log')
    plt.savefig("plots/1/bar_steps.png")
    plt.show()

# 5. Number of evaluations

In [ ]:
with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    plot_bars(stats_df, [*LS_METHODS, *LS_NN_METHODS, *RANDOM_METHODS], selected_instances, "evaluations_mean", ax, "evaluations_std")
    ax.set_ylabel("Number of evaluations")
    ax.set_xlabel("Instance")
    plt.yscale('log')
    plt.tight_layout()
    plt.savefig("plots/1/bar_evaluations.png")
    plt.show()

# 6. Quality Improvement

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(12, 4))#, sharex=True, sharey=True)

for instance, ax in zip(selected_instances, axs.flatten()):
    sns.scatterplot(
        x='initial_cost',
        y='cost',
        hue='method',
        data=all_res_df[(all_res_df['instance'] == instance) & (all_res_df['method'].isin(LS_METHODS))],
        ax=ax,
        palette=COLORS,
        alpha=0.5,
    )
    ax.set_xlabel("Initial cost")
    ax.set_ylabel("Final cost")
    ax.set_title(instance)
    ax.get_legend().remove()

plt.tight_layout()
plt.savefig("plots/1/scatter_initial_final_cost_random_init.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 7))#, sharex=True, sharey=True)
line_colors = sns.color_palette("muted", len(selected_methods))

for instance, ax in zip(selected_instances, axs.flatten()):
    subset_df = all_res_df[(all_res_df['instance'] == instance) & (all_res_df['method'].isin(LS_NN_METHODS))]
    sns.scatterplot(
        x='initial_cost',
        y='cost',
        hue='method',
        data=subset_df,
        ax=ax,
        palette=COLORS,
        alpha=0.5
    )
    
    for i, (method, method_df) in enumerate(subset_df.groupby('method')):
        x = method_df['initial_cost']
        y = method_df['cost']
        coef = np.polyfit(x, y, 1)
        poly1d_fn = np.poly1d(coef)
        # add box with correlation
        ax.text(0.02, 0.95 - 0.05 * i, f"{method}: {np.corrcoef(x, y)[0, 1]:.2f}", transform=ax.transAxes, fontsize=8)

    ax.set_title(instance)
    ax.get_legend().remove()
    ax.set_xlabel("Initial cost")
    ax.set_ylabel("Final cost")

plt.tight_layout()
plt.savefig("plots/1/scatter_initial_final_cost_nn_init.png")
plt.show()

# 7. Number of restarts

In [ ]:
running_cost_df = all_res_df[all_res_df["method"].isin(LS_METHODS + LS_NN_METHODS)].reset_index(drop=True)

running_min_cost = np.zeros(len(running_cost_df),)
running_avg_cost = np.zeros(len(running_cost_df),)

for instance in selected_instances:
    for method in LS_METHODS + LS_NN_METHODS:
        idx = running_cost_df[(running_cost_df["instance"] == instance) & (running_cost_df["method"] == method)].index
        running_min_cost[idx] = running_cost_df[(running_cost_df["instance"] == instance) & (running_cost_df["method"] == method)]["cost"].cummin()
        running_avg_cost[idx] = running_cost_df[(running_cost_df["instance"] == instance) & (running_cost_df["method"] == method)]["cost"].expanding().mean()

running_cost_df["running_min_cost"] = running_min_cost
running_cost_df["running_avg_cost"] = running_avg_cost
# make a column that counts consecutive instance-method pairs
running_cost_df["count"] = running_cost_df.groupby(['instance', 'method']).cumcount()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(12, 6))

for instance, ax in zip(selected_instances, axs.flatten()):
    subset_df = running_cost_df[(running_cost_df["instance"] == instance) & running_cost_df["method"].isin(LS_METHODS + LS_NN_METHODS)]

    
    sns.lineplot(
        x=subset_df["count"],
        y=subset_df['running_min_cost'],
        hue=subset_df["method"],
        ax=ax,
        palette=COLORS,
    )
    sns.lineplot(
        x=subset_df["count"],
        y=subset_df['running_avg_cost'],
        hue=subset_df['method'],
        ax=ax,
        palette=COLORS,
        linestyle='dotted',
    ) 
    ax.set_ylabel("Cost")
    ax.set_xlabel("Iteration")
    
    ax.set_title(instance)
    ax.get_legend().remove()

legend_elements = []
for method in LS_METHODS + LS_NN_METHODS:
    legend_elements.append(plt.Line2D([0], [0], color=COLORS[method], label=f"{method} (min)"))
    legend_elements.append(plt.Line2D([0], [0], color=COLORS[method], linestyle='dotted', label=f"{method} (avg)"))

fig.legend(
    handles=legend_elements,
    loc='upper right',
    bbox_to_anchor=(1.175, 0.95),
)
plt.tight_layout()
plt.savefig("plots/1/line_running_cost.png")
plt.show()

# 8. Best solutions

In [ ]:
best_costs = all_res_df.groupby("instance")["cost"].min()
best_entries = all_res_df[all_res_df["cost"].isin(best_costs)].groupby("instance").first()

best_orders = best_entries.order.to_dict()
best_costs = best_entries.cost.to_dict()

best_dict = {
    k: {"order": best_orders[k], "cost": best_costs[k]} for k in best_orders
}

In [ ]:
def similarity(order_a: list[int], order_b: list[int]) -> float:
    edges_a = set(zip(order_a, [*order_a[1:], order_a[0]]))
    edges_b = set(zip(order_b, [*order_b[1:], order_b[0]]))
    return len(edges_a & edges_b) / len(order_a)

all_res_df["similarity"] = all_res_df.apply(lambda row: similarity(row["order"], best_dict[row["instance"]]["order"]), axis=1)

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(12, 12), sharey=True)

for instance, ax in zip(selected_instances, axs.flatten()):
    subset_df = running_cost_df[(running_cost_df["instance"] == instance) & running_cost_df["method"].isin(LS_METHODS + LS_NN_METHODS)]
    sns.scatterplot(
        x='cost',
        y='similarity',
        hue='method',
        data=all_res_df[(all_res_df['instance'] == instance) & (all_res_df['method'].isin(["steepest-search", "greedy-search", "steepest-search-nn", "greedy-search-nn"]))],
        ax=ax,
        palette=COLORS,
        alpha=0.5,
    )
    ax.set_title(instance)
    ax.get_legend().remove()
    ax.set_xlabel("cost")
    ax.set_ylabel("similarirty")

plt.legend()
plt.tight_layout()
plt.savefig("plots/1/scatter_cost_similarity.png")
plt.show()